# Rossman Store Sales

In [1]:
# Importing necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
# importing train and test dataset
train = pd.read_csv("../Rossmann_Store_Sales/Data/train.csv")
test = pd.read_csv("../Rossmann_Store_Sales/Data/test.csv")
store = pd.read_csv("../Rossmann_Store_Sales/Data/store.csv")

/home/mstevankovic/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#how many datas are in the files:
print("in the training set we have", train.shape[0], "observations and", train.shape[1], "columns/variables.")
print("in the testing set we have", test.shape[0], "observations and", test.shape[1], "columns/variables.")
print("in the store set we have", store.shape[0], "observations and", store.shape[1], "columns/variables.")

in the training set we have 1017209 observations and 9 columns/variables.
in the testing set we have 41088 observations and 8 columns/variables.
in the store set we have 1115 observations and 10 columns/variables.


In [4]:
# the first and the last 5 rows of a train dataset
train.head().append(train.tail())

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1
1017208,1115,2,2013-01-01,0,0,0,0,a,1


In [17]:
# inspecting missing values
train.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [16]:
len(train[(train['Open'] == 1) & (train['Sales'] == 0)])

54